#### Important: Read Before Running

This notebook makes changes to agent and scenario branches indicated in the settings section. Ensure any changes to the target branches are saved prior to running this code. Sedaro recommends committing current changes and creating new branches in the target repositories to avoid loss of work.

This notebook also requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
  "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been
compromised, you can revoke it in the user settings interface on the Sedaro website.

This notebook also requires specific configuration of the HOST and SCENARIO_BRANCH_ID by defining them below or by
including the following in `config.json`

```json
{
  "HOST": "<HOST>",
  "EXAMPLES": {"ROUTINES": {"SCENARIO_BRANCH_ID": "<BRANCH ID>"}}
}


In [ ]:
import json

AGENT_TEMPLATE_BRANCH_ID = ""
HOST = ""

with open('../secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']

try:
    with open('../config.json', 'r') as file:
        config = json.load(file)
        AGENT_TEMPLATE_BRANCH_ID = AGENT_TEMPLATE_BRANCH_ID or config['EXAMPLES']['ROUTINES']['AGENT_TEMPLATE_BRANCH_ID']
        HOST = config['HOST']
except FileNotFoundError:
    pass
except KeyError as e:
    raise KeyError(f"Missing key in config.json: {e}")
finally:
    assert AGENT_TEMPLATE_BRANCH_ID, "AGENT_TEMPLATE_BRANCH_ID must be set if not present in config.json if not set directly."
    assert HOST, "HOST must be set if not present in config.json if not set directly."

#### Branch


In [3]:
from sedaro import SedaroApiClient
sedaro = SedaroApiClient(api_key=API_KEY, host=HOST)
agent_template = sedaro.agent_template(AGENT_TEMPLATE_BRANCH_ID)

#### Example Setup

The following cell creates two generic `Routine` blocks and two conditions as placeholders for the code examples later in this notebook.


In [4]:
example_routine_1 = agent_template.Routine.create(name="Example Routine 1")
example_routine_2 = agent_template.Routine.create(name="Example Routine 2")
example_condition_1 = agent_template.TimeCondition.create(
    name="Example Condition 1", relationship="GREATER", scalar=60000)
example_condition_2 = agent_template.ElapsedTimeCondition.create(
    name="Example Condition 2", relationship="GREATER", scalar={'hour': 1})

## Routines

Before Sedaro 4.9, the internal logic of agents was mostly driven by operational modes. These provided significant utility, but they also had limitations, such as the inability to have two operational modes active simultaneously or to drive the selection of the active operational mode with logic other than conditions and priority. Since the release of 4.9, the functionality of `OperationalMode` has been replaced by `Routine`, which allows for much more comprehensive, complicated, and customizable behavior for agents in Sedaro. Routines are similar to operational modes in that they are used to drive important behavioral logic like active modes and interfaces, but they come in different types that each have their own functionality.

#### `Routine` Types

- [`Routine`](#generic-routine): the generic `Routine` block
- [`CombinationalLogic`](#combinationallogic-routines): A specialized `Routine` block that is associated with various prioritized `LogicalConfiguration` options and re-creates the functionality of operational modes
- [`Schedule`](#schedule-routines): A specialized `Routine` block with a list of subroutines and times that determines when each subroutine will be active
- [`FiniteStateMachine`](#finitestatemachine-routines): A specialized `Routine` block that is associated with `StateTransition` blocks for transitions between subroutines

#### `Routine` Terminology

- A specialized `Routine` chooses between its `subroutines`.
- A set of associated routines with a common ancestor (which itself does not have any parent routines) is called a "program".

#### `Routine` Behavior

- A specialized `Routine` can have at most one active subroutine.
- Otherwise, more than one `Routine` can be active at a time.
- Some behaviors are controlled by the combination of active `Routine`s within a single program. For example, since only one pointing mode can be active at a time, only one `Routine` program may control the agent's active pointing mode.


### Generic `Routine`

The generic `Routine` block is the most basic type of routine. The `Routine` block is used as a leaf subroutine for the more specialized types of routines (`CombinationalLogic`, `Schedule`, and `FiniteStateMachine`) and itself cannot have `subroutines`.

#### `Routine` Example


In [ ]:
generic_routine = agent_template.Routine.create(name="Generic Routine")

### `CombinationalLogic` Routines

Each subroutine of a `CombinationalLogic` is associated with a `LogicalConfiguration` block. Each `LogicalConfiguration` can be defined by a list of conditions, priority, a minimum occurrence duration, maximum occurrence duration, and a minimum time between occurrences. The active subroutine is the one whose `LogicalConfiguration` has the highest `priority` and is compliant. This is equivalent to the behavior of Operational Modes except that any subroutine may optionally be designated as the default subroutine, becoming active when no associated `LogicalConfiguration`s are compliant. 

#### `CombinationalLogic` Example


In [5]:
logical_configuration_1 = agent_template.LogicalConfiguration.create(
    conditions=[example_condition_1.id],
    priority=1,
    routine=example_routine_1.id
)
logical_configuration_2 = agent_template.LogicalConfiguration.create(
    conditions=[example_condition_2.id],
    priority=2,
    routine=example_routine_2.id
)

combinational_logic = agent_template.CombinationalLogic.create(
    name="Combinational Logic Routine",
    logicalConfigurations=[logical_configuration_1.id, logical_configuration_2.id]
)

### `Schedule` Routines

A `Schedule` routine allows you to plan when each of its `subroutines` will be active by defining a set of start and stop times for each subroutine. Like any other kind of `Routine`, only one child routine can be active at a given time.

There are two kinds of `Schedule`: `FixedSchedule` and `RelativeSchedule`. A `FixedSchedule` has the times in its `activeTimes` field set with MJD, which is the default unit of time in Sedaro. A `RelativeSchedule` has the times in its `activeTimes` field set with time since the simulation start time (with a variety of time units available).

#### `FixedSchedule` Example


In [6]:
fixed_schedule = agent_template.FixedSchedule.create(
    name="Fixed Schedule Routine",
    scheduleData={
        example_routine_1.id: {'activeTimes': [(60000, 60000.1), (60000.2, 60000.3)]},
        example_routine_2.id: {'activeTimes': [(60000.4, 60000.5), (60000.6, 60000.7)]}
    },
    subroutines=[example_routine_1.id, example_routine_2.id]
)

#### `RelativeSchedule` Example


In [7]:
relative_schedule = agent_template.RelativeSchedule.create(
    name="Relative Schedule Routine",
    scheduleData={
        example_routine_1.id: {'activeTimes': [({'min': 1}, {'min': 2}), ({'min': 5}, {'min': 10})]},
        example_routine_2.id: {'activeTimes': [({'min': 20}, {'min': 30}), ({'min': 60}, {'min': 120})]}
    },
    subroutines=[example_routine_1.id, example_routine_2.id]
)

### `FiniteStateMachine` Routines

A `FiniteStateMachine` has a set of `StateTransition` blocks that can transition from one `activeSubroutine` to another. Each `StateTransition` has a set of `conditions` that must all be true for the transition from the `fromState` to the `toState` to occur. The `conditions` of a `StateTransition` block with a higher `priority` are checked before `StateTransition` blocks with a lower `priority`. Unlike other types of `Routine`, the `activeSubroutine` field on the `FiniteStateMachine` block needs to be set to indicate its initial state.

#### `FiniteStateMachine` Example


In [9]:
transition_1 = agent_template.StateTransition.create(
    conditions=[example_condition_1.id],
    priority=1,
    fromState=example_routine_1.id,
    toState=example_routine_2.id
)
transition_2 = agent_template.StateTransition.create(
    conditions=[example_condition_2.id],
    priority=2,
    fromState=example_routine_2.id,
    toState=example_routine_1.id
)

finite_state_machine = agent_template.FiniteStateMachine.create(
    name="Finite State Machine Routine",
    transitions=[transition_1.id, transition_2.id],
    activeSubroutine=example_routine_1.id
)